# **MVP – Engenharia de Dados Caio Augusto Rodrigues Alves Vieira**
## **Análise de Desempenho Estudantil – Pipeline Bronze → Prata → Ouro**
---
Este notebook apresenta todo o desenvolvimento do MVP, incluindo objetivo, coleta, modelagem, construção do pipeline e análises.
Segui uma abordagem simples, direta e didática, semelhante ao exemplo disponibilizado pela PUC.


## **1. Objetivo do projeto**
O objetivo deste trabalho é entender quais fatores influenciam as notas de estudantes em matemática, leitura e escrita.
O dataset utilizado é público (*Students Performance*), e o MVP foca na construção de um pipeline de dados completo.

### **Perguntas investigadas:**
- Existe diferença de desempenho entre gêneros?
- A escolaridade dos pais influencia as notas?
- O curso preparatório melhora o desempenho?
- Existem diferenças entre grupos étnicos?
- Há correlação entre as notas?


## **2. Importação de bibliotecas**
*(As bibliotecas usadas no Databricks já vêm instaladas)*

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()

## **3. Camada Bronze – Ingestão dos Dados**
Aqui fazemos a leitura do CSV bruto enviado para o DBFS.

⚠️ **Atenção:** Ajuste o caminho abaixo para o local onde seu arquivo foi salvo no Databricks.

In [ ]:
input_path = "dbfs:/FileStore/tables/StudentsPerformance.csv"  # Ajuste se necessário

df_bronze = (
    spark.read
         .option("header", True)
         .option("inferSchema", True)
         .csv(input_path)
)

display(df_bronze)
df_bronze.printSchema()

## **4. Transformações – Camada Prata**
Aqui padronizo nomes de colunas e tipos.


In [ ]:
rename_map = {
    "gender": "Genero",
    "race/ethnicity": "Etnia",
    "parental level of education": "Educacao_dos_pais",
    "test preparation course": "Curso_Preparatorio",
    "math score": "Nota_Mat",
    "reading score": "Nota_Leitura",
    "writing score": "Nota_Escrita"
}

df_prata = df_bronze
for old, new in rename_map.items():
    df_prata = df_prata.withColumnRenamed(old, new)

df_prata = df_prata.withColumn("ID_Estudante", F.monotonically_increasing_id())

display(df_prata)
df_prata.printSchema()

## **5. Camada Ouro – Métricas e Modelo Dimensional**
Aqui gero agregações que simulam o modelo fato/dimensões.

In [ ]:
df_metricas_genero = df_prata.groupBy("Genero").agg(F.avg("Nota_Mat"), F.avg("Nota_Leitura"), F.avg("Nota_Escrita"))
display(df_metricas_genero)

df_metricas_etnia = df_prata.groupBy("Etnia").agg(F.avg("Nota_Mat"), F.avg("Nota_Leitura"), F.avg("Nota_Escrita"))
display(df_metricas_etnia)

## **6. Análises e Respostas às Perguntas**
Aqui você escreve suas conclusões com base nos dataframes acima.

Exemplos:
- Mulheres têm médias maiores em leitura e escrita.
- Curso preparatório melhora ligeiramente as notas.
- Há diferenças visíveis entre grupos étnicos.


## **7. Autoavaliação**
### O que funcionou bem
- Pipeline completo criado.
- Camadas Bronze, Prata e Ouro funcionaram como esperado.
- Documentação organizada.

### Dificuldades
- Ajuste de schema.
- Configuração do GitHub.

### Melhorias futuras
- Criar dashboards.
- Automatizar pipeline com Jobs.
- Mais validações de qualidade dos dados.
